# 資料準備

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


from tensorflow.examples.tutorials.mnist import input_data

import sys
sys.path.append('../')
from util.my_plot import layer,plot_images_labels_prediction, plot_batch, generate_shift_fashion_mnist_data

In [ ]:
fashion_mnist = input_data.read_data_sets('input/data', one_hot=True)


# 建立模型

In [ ]:
# todo: 
# 建立輸入層  

In [ ]:
# todo: 
# 建立隱藏層h1

In [ ]:
# todo: 
# 建立輸出層

# 定義訓練方式

In [ ]:
#建立訓練資料label真實值 placeholder

In [ ]:
y_label = tf.placeholder("float", [None, 10])

In [ ]:
# todo:
#定義loss function

In [ ]:
# todo:
# 選擇optimizer

# 定義評估模型的準確率

In [ ]:
# todo: 
# 定義評估graph

# 開始訓練

In [ ]:
trainEpochs = 15
batchSize = 100
totalBatchs = int(fashion_mnist.train.num_examples/batchSize)
epoch_list=[]
val_loss_list=[];val_accuracy_list=[]
loss_list=[];accuracy_list=[]
from time import time
startTime=time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
print(totalBatchs)

In [ ]:
# 進行多次實驗
for epoch in range(trainEpochs):
    # 每一步取一批做運算
    for i in range(totalBatchs):
        batch_x, batch_y = fashion_mnist.train.next_batch(batchSize)
        sess.run(optimizer,feed_dict={x: batch_x/255 ,y_label: batch_y})
    
    # 計算training data的準確度 
    loss,acc = sess.run([loss_function,accuracy], 
                        feed_dict={x: fashion_mnist.train.images/255, 
                                   y_label: fashion_mnist.train.labels})

    
    # 計算validate data的準確度 
    val_loss,val_acc = sess.run([loss_function,accuracy], 
                                feed_dict={x: fashion_mnist.validation.images/255, 
                                           y_label: fashion_mnist.validation.labels})

    
    epoch_list.append(epoch)
    loss_list.append(loss);accuracy_list.append(acc)    
    val_loss_list.append(val_loss);val_accuracy_list.append(val_acc)    
    print("Train Epoch:", '%02d' % (epoch+1), "Loss=", "{:.9f}".format(loss), " Accuracy=",acc ,"val Loss=", "{:.9f}".format(val_loss)," val_Accuracy=",val_acc  )
    
duration =time()-startTime
print("Train Finished takes:",duration)      

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.gcf()
fig.set_size_inches(4,2)
plt.plot(epoch_list, loss_list, label = 'loss')
plt.plot(epoch_list, val_loss_list, label = 'val_loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')


In [ ]:
plt.plot(epoch_list, accuracy_list,label="accuracy" )
plt.plot(epoch_list, val_accuracy_list,label="accuracy" )

fig = plt.gcf()
fig.set_size_inches(4,2)
plt.ylim(0,1)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# 評估模型準確率

In [ ]:
print("Accuracy:", sess.run(accuracy,
                           feed_dict={x: fashion_mnist.test.images/255,
                                      y_label: fashion_mnist.test.labels}))

# 進行預測

In [ ]:
prediction_result=sess.run(tf.argmax(y_predict,1),
                           feed_dict={x: fashion_mnist.test.images/255 })

In [ ]:
prediction_result[:10]

In [ ]:
fashion_mnist.test.images.shape
aa = fashion_mnist.test.images.reshape(10000,28,28)
prediction_result.shape
fashion_mnist.test.labels

In [ ]:
plot_images_labels_prediction(aa,
                              np.argmax(fashion_mnist.test.labels,axis=1),
                              prediction_result,0)

In [ ]:
y_predict_Onehot=sess.run(y_predict,
                          feed_dict={x: fashion_mnist.test.images/255 })

In [ ]:
y_predict_Onehot[8]

In [ ]:
from util.my_plot import generate_shift_mnist_data
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

shift_img, shift_img_label = generate_shift_fashion_mnist_data(10000)
plot_batch(shift_img, shift_img_label,class_names, 5)

In [ ]:
from keras.utils import np_utils
bb = shift_img.reshape(10000, 784)
shift_img_label_OneHot = np_utils.to_categorical(shift_img_label)

In [ ]:
print("Accuracy:", sess.run(accuracy,
                           feed_dict={x: bb/255,
                                      y_label: shift_img_label_OneHot}))

In [ ]:
sess.close()